Preliminary Data Analysis of Mozilla SurveyGizmo Feedback Data

Note: must also install vaderSentiment (pip install vaderSentiment), and assumes that files 'Top Sites for Report Analysis.csv' and '20180912113810-SurveyExport.csv' are located in the same directory as the file

In [ ]:
import pandas as pd
import re
from constants import WORDS_TO_COMPONENT, WORDS_TO_ISSUE 
from whoosh.analysis import *
#Some very basic visualizations
import ipywidgets as widgets

In [2]:
# for columns A and B in the top 100, get strings in cells, comma, split by comma, 
#then save and check if data contains these values
sites = pd.read_csv('C:/Users/amritpd/Downloads/Top Sites for Report Analysis.csv', usecols=['Domains','Brand'])
#, skiprows = 50, nrows = 25
#display(sites)
siteList = list(sites.values.flatten())
issuesFile = open('issues.txt', 'r')
issuesList = issuesFile.read().split(',')

#remove commas ('salesforce.com, force.com')
for site in siteList:
    if ',' in site:
        siteList += site.split(',')
        
siteList = [site.strip('.*') for site in list(filter(lambda site: ',' not in site, siteList))]
issuesList = [issue.strip('\n') for issue in issuesList]

#print(issuesList)



In [38]:
#Clean up the raw dictionaries a bit more

#FOR NOW just lower the terms in the lists. Need to see how steming and more can play into this
WORDS_TO_ISSUE = {k: v.lower() for k, v in WORDS_TO_ISSUE.items()}
#Filter we will initially use 
stmLwrFilter = RegexTokenizer() | StemFilter() | LowercaseFilter() 
lwrFilter = LowercaseFilter()
display(WORDS_TO_ISSUE.items())




dict_items([('speed related', ['slow', 'fast', 'lag', 'laggy', 'quick', 'speed', 'smooth']), ('crash related', ['crash', ' frozen', 'abort', 'abandon', 'freeze', 'dead', 'died']), ('UI', ['pretty', 'beautical', 'ugly', ' elegant', 'good looking', 'good-looking', 'nice', 'pleasant', 'awful', 'great', 'UI', 'user interface', 'design']), ('Update', ['update', 'release', 'updated', 'new version']), ('Compatibility', ['compatible']), ('Bug', ['doesn’t', 'doesnot', 'does not', "didn't", 'did not', "don't", ' donot', ' do not', " can't", ' cannot', ' can not', "couldn't", ' couldnot', ' could not', ' will not', " won't", " wouldn't", ' would not', ' wouldnot', " isn't", ' not', ' are not', ' were not', ' was not', ' click', 'work', 'working', 'run', 'delete', 'save', 'find', 'open', 'close', 'turnon', 'turnoff', 'remove', 'clear', 'hide', 'add', 'edit', 'change', 'view', 'see', 'erase', 'move', 'enable', 'disable', 'login', 'log in', 'logon', ' log on', ' search', 'exit', 'leave', 'bookmark',

In [3]:
fields = ['KI', 'KN', 'KP']
#read in raw survey data from CSV files. Only want certain columns
survey_cols = ["Response ID","Time Started","Date Submitted","Status","Language","Referer","Extended Referer","User Agent","Extended User Agent","Longitude","Latitude","Country","City","State/Region","Postal","How does Firefox make you feel?","OS","To help us understand your input, we need more information. Please describe what you like. The content of your feedback will be public, so please be sure not to include personal information such as email address, passwords or phone number.","To help us understand your input, we need more information. Please describe your problem below and be as specific as you can. The content of your feedback will be public, so please be sure not to include personal information such as email address, passwords or phone number.","If your feedback is related to a website, you can include it here:"]
df = pd.read_csv("C:/Users/amritpd/Downloads/20180928025007-SurveyExport.csv", encoding = "ISO-8859-1", nrows=6000, usecols=survey_cols)
#some data cleaning and selection 
#rename some long column names
df.rename(columns={ survey_cols[15]: 'Binary Sentiment', survey_cols[17]: 'Positive Feedback', survey_cols[18]: 'Negative Feedback', survey_cols[19]: 'Relevant Site'}, inplace=True)
df = df.fillna(''); #repalce NaNs with blanks
df = df.loc[df['Status'] == 'Complete'] #Only want completed surveys
df = df.loc[df['Language'] == 'English'] #Only want english rows
#Convert to df friendly date-times
df["Date Submitted"] = pd.to_datetime(df["Date Submitted"])
df["Time Started"] = pd.to_datetime(df["Time Started"])


if df.empty: #need to handle empty case later
    print('DataFrame is empty!')
else:
    print('Not empty!')



Not empty!


In [4]:
#Sort df by latest submitted data 
df = df.sort_values(by="Date Submitted", ascending=0)
display(df.tail(5))

,Response ID,Time Started,Date Submitted,Status,Language,Referer,Extended Referer,User Agent,Extended User Agent,Longitude,Latitude,Country,City,State/Region,Postal,Binary Sentiment,OS,Positive Feedback,Negative Feedback,Relevant Site
5,134,2016-11-25 13:29:59,2016-11-25 13:30:07,Complete,English,,,Mozilla/5.0 (Android 7.0; Mobile; rv:50.0) Gec...,Mozilla/5.0 (Android 7.0; Mobile; rv:50.0) Gec...,-122.682701,45.533199,United States,Portland,OR,97209,Happy,,Asdf,,
4,133,2016-11-25 13:28:57,2016-11-25 13:29:58,Complete,English,,,Mozilla/5.0 (Android 7.0; Mobile; rv:50.0) Gec...,Mozilla/5.0 (Android 7.0; Mobile; rv:50.0) Gec...,-122.682701,45.533199,United States,Portland,OR,97209,Happy,,Asdf,,
2,129,2016-11-25 13:13:11,2016-11-25 13:20:59,Complete,English,,,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; ...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; ...,-122.682701,45.533199,United States,Portland,OR,97209,Sad,,asfasdfas,,
1,128,2016-11-23 15:04:02,2016-11-23 15:12:29,Complete,English,,,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; ...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; ...,-122.643097,45.544201,United States,Portland,OR,97212,Sad,,asdfasdf,,
0,124,2016-11-15 10:55:50,2016-11-15 10:56:48,Complete,English,,,Mozilla/5.0 (Android 7.0; Mobile; rv:49.0) Gec...,Mozilla/5.0 (Android 7.0; Mobile; rv:49.0) Gec...,-122.682701,45.533199,United States,Portland,OR,97209,Sad,,Askd,,


In [5]:
#Derive certain columns
#start with basic sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

analyzer = SIA()
results = []

#just append & analyze the -ve/+ve feedback for now if user gave both
#df[['Neg', 'Neu', 'Pos', 'Compound']] = df['Text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in df.iterrows(): 
    pol_score = analyzer.polarity_scores(row['Positive Feedback'] + row['Negative Feedback'])
    results.append(pol_score)

df2 = pd.DataFrame.from_records(results)    
# df['Neg'], df['Neu'], df['Pos'], 'Compound']] = results

df = pd.merge(df, df2, left_index=True, right_index=True)

C:\Anaconda\envs\mie451-lab3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [45]:
#Filter we will initially use 
#stmLwrFilter = RegexTokenizer() | StemFilter() | LowercaseFilter() 

#Derive NLP version of raw feedback column 
def apply_nlp (series):
    combined = series['Positive Feedback'] + series['Negative Feedback']
    filtered = [token.text for token in stmLwrFilter(combined)]
    return list(set(filtered)) #turn to set temporarily to get unique values 

#crude way of looking for mentioned site using the top 100 list. Need to add the regex to pick up wildcard sites
def mentionedSite (series):
    combined = series['Relevant Site'] + series['Positive Feedback'] + series['Negative Feedback']
    sites = [site.lower() for site in siteList if site.lower() in combined.lower()]
    return sites 

#Find a mentioned issue based on our issues dictionary 
def mentionedIssue (series):
    combined = series['Positive Feedback'] + series['Negative Feedback']
    issues = [k for k, v in WORDS_TO_ISSUE.items() if any(map(lambda term: term in combined.lower(), v))]
    return issues

def mentionedComponent (series):
    combined = series['Positive Feedback'] + series['Negative Feedback']
    issues = [k for k, v in WORDS_TO_COMPONENT.items() if any(map(lambda term: term in combined.lower(), v))]
    return issues

print(hi)

test = " lag slow"
hi = [k for k, v in WORDS_TO_ISSUE.items() if any(map(lambda term: term in test, v))]

    

['speed related']


In [54]:
#See if any of the top sites are mentioned in a list, see if any issues are mentioned 
df['Processed Feedback'] = df.apply(apply_nlp, axis=1)
df['Sites'] = df.apply(mentionedSite, axis=1 )
df['Issues'] = df.apply(mentionedIssue, axis=1)
df['Components'] = df.apply(mentionedComponent, axis=1)

In [55]:
display(df.tail(40))

type( df.iloc[0]["Processed Feedback"] )

,Response ID,Time Started,Date Submitted,Status,Language,Referer,Extended Referer,User Agent,Extended User Agent,Longitude,...,Negative Feedback,Relevant Site,compound,neg,neu,pos,Sites,Issues,Processed Feedback,Components
53,595,2016-12-19 13:43:57,2016-12-19 13:45:24,Complete,English,,,Mozilla/5.0 (Android 5.1; Tablet; rv:49.0) Gec...,Mozilla/5.0 (Android 5.1; Tablet; rv:49.0) Gec...,-76.888397,...,,,-0.1531,0.211,0.789,0.000,[],[],[w],[Preferences]
52,575,2016-12-19 13:42:54,2016-12-19 13:44:17,Complete,English,,,Mozilla/5.0 (Android 5.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 5.0.1; Mobile; rv:50.0) G...,-117.440804,...,,,0.0000,0.000,1.000,0.000,[],[Bug],"[my, me, in, tab, s, find, for, of, easi, them...",[Preferences]
51,574,2016-12-19 13:42:49,2016-12-19 13:43:26,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,-0.016700,...,,,0.0000,0.000,1.000,0.000,[],[Bug],"[in, the, tab, of, lot, background, opene]",[Preferences]
50,569,2016-12-19 13:42:15,2016-12-19 13:43:14,Complete,English,,,Mozilla/5.0 (Android 6.0; Mobile; rv:50.0) Gec...,Mozilla/5.0 (Android 6.0; Mobile; rv:50.0) Gec...,-117.133598,...,,,0.7269,0.000,0.757,0.243,[],[],"[best, browser, evaaaa]",[Preferences]
49,565,2016-12-19 13:41:58,2016-12-19 13:42:55,Complete,English,,,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) G...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) G...,-5.302800,...,ANNOYING POP UPS FROM FIREFOX are stopping som...,,0.0000,0.000,1.000,0.000,[],[Bug],"[annoying, ups, ar, pop, some, stopp, firefox,...",[Preferences]
48,558,2016-12-19 13:41:30,2016-12-19 13:42:00,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,-81.409698,...,,stop this,0.0000,0.000,1.000,0.000,[],[],[no],[Preferences]
47,532,2016-12-19 13:37:57,2016-12-19 13:38:34,Complete,English,,,Mozilla/5.0 (Android 5.1; Mobile; rv:50.0) Gec...,Mozilla/5.0 (Android 5.1; Mobile; rv:50.0) Gec...,-118.189201,...,,,0.3612,0.000,0.615,0.385,[],[],[secur],[Preferences]
46,529,2016-12-19 13:37:18,2016-12-19 13:38:13,Complete,English,,,Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) ...,Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) ...,-84.324402,...,https://www.bloomberg.com/ can't run videos,https://www.bloomberg.com/,0.0000,0.000,1.000,0.000,[],[Bug],"[t, run, www.bloomberg.com, video, http, can]",[Preferences]
45,517,2016-12-19 13:36:15,2016-12-19 13:36:30,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,-93.378601,...,Slow,,-0.2755,0.105,0.895,0.000,[],[speed related],[slow],"[Slowness, Preferences]"
43,480,2016-12-19 13:32:37,2016-12-19 13:34:14,Complete,English,,,Mozilla/5.0 (Android 4.4; Mobile; rv:47.0) Gec...,Mozilla/5.0 (Android 4.4; Mobile; rv:47.0) Gec...,51.330399,...,,,0.0000,0.000,1.000,0.000,[gmail],[],"[gmail.com, shaheen1343]",[Preferences]


list

In [ ]:
#Some very basic visualizations
import ipywidgets as widgets

#quick overview of the number of sads/happies over time. You can notice a trend. 
start_date = df["Date Submitted"].min()
end_date = df["Date Submitted"].max()
display("Most Recent Date: ", end_date)

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

slide = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)
#Trends over time. Look at binary sentiment for a period of time, most -ve comments at that time
def binary_sentiment(slider):
    subset = df[(df['Date Submitted'] >= slider[0]) & (df['Date Submitted'] <= slider[1])]
    display("Happy-Sadness counts in this time interval: ", subset["Binary Sentiment"].value_counts())
        
widgets.interactive(binary_sentiment,slider=slide)
# I noticed a bunch of new feedback after December 19th 2016.
# This could be because of Mozilla's latest release of V50.1 earlier in the week.
# Probably introduced some breaking changes. 

In [94]:
#Select top site, show relevant comments sorted by most negative and most recent
topSites = ['facebook', 'yahoo', 'google', 'twitter', 'netflix']
issues = list(WORDS_TO_ISSUE.keys())
components = list(WORDS_TO_COMPONENT.keys())

# df_issues = df[df['Issues'] != []]

# df_components = df[df['Components'] != [] ]
print("ISSUE & COMPONENT SUMMARY STATISTICS:\n\n")
print("In the last few weeks, there has been " + str(len(df.index)) + " user feedback points.\n\n") 
print("Out of these, " + str(len( df['Issues'].notnull().index )) + " were related to an issue and " + str(len( df['Components'].notnull().index )) + " were related to a component/feature.\n\n") 


,Response ID,Time Started,Date Submitted,Status,Language,Referer,Extended Referer,User Agent,Extended User Agent,Longitude,...,Negative Feedback,Relevant Site,compound,neg,neu,pos,Sites,Issues,Processed Feedback,Components
3713,38008,2016-12-22 20:20:50,2016-12-22 20:21:15,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,-122.676201,...,,,0.0000,0.000,1.000,0.000,[],[],[2134456546],[Preferences]
3712,38003,2016-12-22 20:20:06,2016-12-22 20:20:43,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:50.0) G...,-96.811699,...,,,0.0000,0.000,1.000,0.000,[],[],"[s, good, it]",[Preferences]
3702,37899,2016-12-22 19:56:44,2016-12-22 20:14:44,Complete,English,,,Mozilla/5.0 (Android 6.0; Mobile; rv:50.0) Gec...,Mozilla/5.0 (Android 6.0; Mobile; rv:50.0) Gec...,3.395800,...,,,0.0000,0.000,1.000,0.000,[],[],[08030535563],[Preferences]
3709,37963,2016-12-22 20:11:58,2016-12-22 20:12:24,Complete,English,,,Mozilla/5.0 (iPhone; CPU iPhone OS 10_2 like M...,Mozilla/5.0 (iPhone; CPU iPhone OS 10_2 like M...,-111.920502,...,Allow Firefox Themes on iOS,,0.0000,0.000,1.000,0.000,[],[],"[theme, allow, firefox, ios, on]",[Preferences]
3707,37957,2016-12-22 19:23:34,2016-12-22 20:12:13,Complete,English,,,Mozilla/5.0 (Android 6.0.1; Mobile; rv:47.0) G...,Mozilla/5.0 (Android 6.0.1; Mobile; rv:47.0) G...,153.024307,...,,,0.0000,0.000,1.000,0.000,[],[],[life],[Preferences]
3706,37926,2016-12-22 20:03:34,2016-12-22 20:05:59,Complete,English,,,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) G...,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) G...,174.807098,...,"your new version 50.1 is totally rubbish, it c...",,0.0000,0.000,1.000,0.000,[],"[crash related, Update, Bug]","[show, ar, in, total, respond, rubbish, keep, ...","[Firefox, Crashes, Preferences]"
3700,37891,2016-12-22 19:54:57,2016-12-22 20:03:00,Complete,English,,,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) G...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) G...,-97.858704,...,"FF Desktop 50.1.0 The ""LastPass.com"" plugin...",,-0.7992,0.086,0.888,0.025,"[google, google maps]",[Bug],"[problem, have, then, in, fill, tab, doe, icon...","[Firefox, Preferences]"
3701,37892,2016-12-22 19:55:19,2016-12-22 19:59:17,Complete,English,,,Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_5 like ...,Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_5 like ...,0.000000,...,ABUSE BY SAFARI TAKING OVER PAGES & APPS PAGES...,"Block ,DELETE SAFARI NOW ,ABUSE TWITTER",0.6187,0.000,0.893,0.107,[twitter],[Bug],"[taking, block, fake, is, safari, twitter, acc...","[Firefox, Preferences]"
3699,37889,2016-12-22 19:54:45,2016-12-22 19:56:54,Complete,English,https://www.mozilla.org/en-US/contact/,https://www.mozilla.org/en-US/contact/,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) G...,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) G...,-157.827393,...,,,0.3802,0.000,0.436,0.564,[],[Bug],"[like, but, check, donat, to, card, don, m, ha...",[Preferences]
3694,37855,2016-12-22 19:48:51,2016-12-22 19:55:35,Complete,English,,,Mozilla/5.0 (Android 5.0.2; Tablet; rv:50.0) G...,Mozilla/5.0 (Android 5.0.2; Tablet; rv:50.0) G...,-123.082497,...,,,0.4404,0.000,0.633,0.367,[],[],"[why, monei, love, i, is, thi, you, that, prog...",[Preferences]


10

ISSUE & COMPONENT SUMMARY STATISTICS:


In the last few weeks, there has been 2332 user feedback points.


Out of these, 2332 were related to an issue and 2332 were related to a component/feature.




In [63]:
tsdrop = widgets.Dropdown(description="Top Sites", options=topSites)
def basic_site_analysis(drop):
    #display the most negatively associated comments
    display("MOST NEGATIVE COMMENTS for " + drop)
    subset = df[df['Relevant Site'].str.contains(drop)]
    display(subset.nsmallest(3, 'compound'))
    display("MOST RECENT COMMENTS for " + drop)
    display(subset.nlargest(3, 'Date Submitted'))


#Select issue, "" "" 
widgets.interactive(basic_site_analysis,drop=tsdrop)
#tsdrop.observe(basic_site_analysis, names='value')

interactive(children=(Dropdown(description='Top Sites', options=('facebook', 'yahoo', 'google', 'twitter', 'ne…

In [ ]:
#elastic search (?) TF-IDF  There will always be some error rate that we can quantify and analyze
#2 years ago team used Tkinter for a basic Python GUI 
#Back end is more important to be usable. The first UI is the quickest for the MVP.
#any one of those tabs will be an MVP (trends and/or search as MVP)
#two fold - broad analysis, moving average, think of statistics at the overall corpus level (histograms)
#top K words wit. How stable are complaints over time, or how are issues/features mentioned changing over time